<h1> Exercise 3.5

In [3]:
data <- read.csv("./data/TrainExer 3-5.csv")
names(data)[1] <- "Observation"

In [4]:
summary(data)

  Observation       Index           Dividends         Riskfree      
 Min.   :1927   Min.   :   6.89   Min.   : 0.440   Min.   :0.00010  
 1st Qu.:1948   1st Qu.:  20.93   1st Qu.: 1.060   1st Qu.:0.00925  
 Median :1970   Median :  92.43   Median : 3.140   Median :0.03060  
 Mean   :1970   Mean   : 336.76   Mean   : 7.391   Mean   :0.03566  
 3rd Qu.:1992   3rd Qu.: 426.40   3rd Qu.:12.290   3rd Qu.:0.05180  
 Max.   :2013   Max.   :1848.36   Max.   :34.990   Max.   :0.15490  
   LogEqPrem          BookMarket          NTIS              DivPrice     
 Min.   :-0.56374   Min.   :0.1312   Min.   :-0.041811   Min.   :-4.478  
 1st Qu.:-0.03466   1st Qu.:0.3577   1st Qu.: 0.008141   1st Qu.:-3.556  
 Median : 0.08037   Median :0.5547   Median : 0.017962   Median :-3.334  
 Mean   : 0.05975   Mean   :0.5737   Mean   : 0.018630   Mean   :-3.354  
 3rd Qu.: 0.20006   3rd Qu.:0.7311   3rd Qu.: 0.027075   3rd Qu.:-3.026  
 Max.   : 0.42221   Max.   :1.4421   Max.   : 0.165576   Max.   :-2.293  

Index -> Index+Dividends -> change in log(Index+Dividends) -> (1-Riskfree)change in log((1+Dividends)*Index) -> LogEqPrem

<h3>(a) Replicate the R^2 values of slide 7 from lecture 3.5. In particular, show that a regression of the log equity premium (the variable LogEqPrem in the data file) on a constant and all five explanatory variables gives an $R^2$ of 10.8%, and that a regression of the log equity premium on a constant and only book-to-market gives an R^2 of 6.3%. Then, based on these values, argue whether the additional four variables are significant when
comparing the full with the book-to-market only model. </h3> 

In [7]:
fit <- lm(LogEqPrem ~ BookMarket + NTIS + DivPrice + EarnPrice + Inflation, data=data)
sprintf("R-squared for full model: %.3f", summary(fit)$r.squared)

[1] "R-squared for full model: 0.108"

In [8]:
fit2 <- lm(LogEqPrem ~ BookMarket, data=data)
sprintf("R-squared for book-to-market model: %.3f", summary(fit2)$r.squared)

[1] "R-squared for book-to-market model: 0.063"

In [19]:
k <- 6 #5 + 1 constant
g <- 4
n <- nrow(data)
Ftest <- (summary(fit)$r.squared - summary(fit2)$r.squared)*(n-k)/g/(1 - summary(fit)$r.squared)
sprintf("F-test value = %.3f", Ftest)
sprintf("p-value = %.3f", 1 - pf(Ftest, g, n-k))

[1] "F-test value = 1.025"

[1] "p-value = 0.399"

P-value > 0.05. Therefore, do not reject $H_0$

<h3>(b) Replicate the RESET statistic of slide 8 of Lecture 3.5. Proceed in the following steps. First regress the log equity premium on a constant and the book-to-market ratio. Then store the fitted log equity premium based on the output from this regression. Finally, regress the log equity premium on a constant, the book-to-market ratio, and the square of the fitted log equity premium that was stored in the previous step. The RESET test statistic is the statistic of an F-test on the fitted log equity premium parameter.

In [35]:
data$Pred <- predict(fit2)**2

In [45]:
fit3 <- lm(LogEqPrem ~ BookMarket + Pred, data=data)

In [46]:
k <- 6 #5 + 1 constant
p <- 1
n <- nrow(data)
Ftest2 <- (sum(summary(fit2)$residuals**2) - sum(summary(fit3)$residuals**2))*(n-k-p)/p/(sum(summary(fit3)$residuals**2))
sprintf("F-test value = %.3f", Ftest2)
sprintf("p-value = %.3f", 1 - pf(Ftest2, p, n-k-p))

[1] "F-test value = 3.282"

[1] "p-value = 0.074"

P-value > 0.05. Therefore, do not reject $H_0$

<h3>(c) Replicate the Chow break statistic of slide 8 of Lecture 3.5. Proceed in the following steps. First regress the log equity premium on a constant and the book-to-market ratio and store the sum of squared residuals.  Then perform the same regression for both the subsample of observations over 1927-1979, and the subsample of observations over 1980-2013. For both regressions, store the sum of squared residuals. Use these sum of squared residuals to calculate the Chow break statistic.
</h3>

In [41]:
e_restricted <- sum(summary(fit2)$residuals**2)

In [49]:
data_1 <- data[data$Observation < 1980,]
fit4 <- lm(LogEqPrem ~ BookMarket, data=data_1)
e_1 <- sum(summary(fit4)$residuals**2)

In [50]:
data_2 <- data[data$Observation >= 1980,]
fit5 <- lm(LogEqPrem ~ BookMarket, data=data_2)
e_2 <- sum(summary(fit5)$residuals**2)

In [51]:
e_unrestricted <- e_1 + e_2

In [54]:
k <- 2
n <- nrow(data)
Ftest3 <- (e_restricted - e_unrestricted)*(n-2*k)/k/(e_unrestricted)
sprintf("F-test value = %.3f", Ftest3)
sprintf("p-value = %.3f", 1 - pf(Ftest3, k, n-2*k))

[1] "F-test value = 2.269"

[1] "p-value = 0.110"

P-value > 0.05. Therefore, do not reject $H_0$

<h3> (d) Replicate the Chow forecast statistic of slide 8 of Lecture 3.5. No new regression is required, you should be able to base this result on the regressions you have run so far </h3>

In [56]:
k <- 2
n_1 <- nrow(data_1)
n_2 <- nrow(data_2)
Ftest4 <- (e_restricted - e_1)*(n_1-k)/n_2/(e_1)
sprintf("F-test value = %.3f", Ftest4)
sprintf("p-value = %.3f", 1 - pf(Ftest4, n_2, n_1-k))

[1] "F-test value = 0.765"

[1] "p-value = 0.794"

P-value > 0.05. Therefore, do not reject $H_0$